In [4]:
# The aim of this script is to download all rental property data and all available property from realestate website.
# the looping method doesn't work, hence the long winded version

# pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api

from realestate_com_au import RealestateComAu
api = RealestateComAu()

# Get property listings
# listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])

import pandas as pd

In [ ]:
# https://www.matthewproctor.com/australian_postcodes
# df_postcode = pd.read_excel (r'/Users/joezhou/Downloads/Realestate/australian_postcodes.xlsx', sheet_name='Sheet1')

# sample test suburb
# df_postcode_subset = df_postcode.loc[(df_postcode['Locality']=='LIDCOMBE') & (df_postcode['Postcode']==2141) ]

# df_postcode_subset["VAR_Find"] = '["'+df_postcode_subset['Postcode'].apply(str)+","+df_postcode_subset['Locality']+'"]'
# list_postcode = list(df_postcode_subset['VAR_Find'].dropna())

In [6]:

df_out = api.search(locations=['2141,LIDCOMBE'],channel="rent")
DF_RENT = DF_RENT.append(df_out, ignore_index=True) 
print("finished rent:2141-LIDCOMBE")


DF_RENT1 = DF_RENT.reset_index()
DF_RENT2 = DF_RENT1.drop(columns=['id','index','images','images_floorplans','listers'])
DF_RENT_UNIQ = DF_RENT2.drop_duplicates()

DF_RENT_UNIQ.to_pickle("/Users/joezhou/Downloads/Realestate/_PCODE_R_RENT_NSW.pkl")

# Everything within the postcode

df_out = api.search(locations=['2141,LIDCOMBE'] )
DF_ALL = DF_ALL.append(df_out, ignore_index=True) 
print("finished all:2141-LIDCOMBE")
df_out = pd.DataFrame(df_out)

DF_ALL1 = DF_ALL.reset_index()
DF_ALL2 = DF_ALL1.drop(columns=['id','index','images','images_floorplans','listers'])
DF_ALL_UNIQ = DF_ALL2.drop_duplicates()

DF_ALL.to_pickle("/Users/joezhou/Downloads/Realestate/_PCODE_R_ALL_NSW.pkl")

KeyboardInterrupt: 

In [ ]:
# consolidate for aggregation comparisons


REF_PCODE_SBRB = INFO_ALL.append([INFO_ALL],ignore_index=True).reset_index() 
# REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb"]]
# full comparison version
REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb","property_type"]]
REF_PCODE_SBRB = REF_PCODE_SBRB.drop_duplicates()

